## test

In [ ]:
import tensorflow as tf
import os

# Set the CUDA device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Check TensorFlow version and available GPU devices
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

## import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Convolution2D, Flatten
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model, Model

## lable preparation

In [ ]:
# Read the training, validation, and test datasets from CSV files
tr_df = pd.read_csv('F:/poorya/datasets/ThermalFaceDatabase/X_train.csv')
vl_df = pd.read_csv('F:/poorya/datasets/ThermalFaceDatabase/X_val.csv')
ts_df = pd.read_csv('F:/poorya/datasets/ThermalFaceDatabase/X_test.csv')

In [ ]:
def df_to_ary(df):
    '''
    Convert Pandas DataFrame columns 'x' and 'y' to a NumPy array.

    Arguments:
    df (pandas.DataFrame): Input DataFrame containing 'x' and 'y' columns.

    Returns:
    numpy.ndarray: NumPy array with transformed values from 'x' and 'y' columns.
    '''
    ary = np.zeros((len(df), 136), dtype = int)

    for i in range(len(df)):
        # Identify comma positions in 'x' and 'y' strings
        rep_x = [j for j in range(len(df['x'][i])) if df['x'][i].startswith(',', j)]
        rep_y = [j for j in range(len(df['y'][i])) if df['y'][i].startswith(',', j)]

        # Extract and store x, y coordinates into the array
        ary[i, 0] = int(df['x'][i][1:rep_x[0]])
        ary[i, 67] = int(df['x'][i][rep_x[-1] + 1:-1])
        ary[i, 68] = int(df['y'][i][1:rep_y[0]]) - 128
        ary[i, 135] = int(df['y'][i][rep_y[-1] + 1:-1]) - 128

        for k in range(66):
            ary[i, k + 1] = int(df['x'][i][rep_x[k] + 1:rep_x[k + 1]])
            ary[i, k + 69] = int(df['y'][i][rep_y[k] + 1:rep_y[k + 1]]) - 128

    return ary


# Convert DataFrame columns 'x' and 'y' to NumPy arrays
tr_ary = df_to_ary(tr_df)  # Training data
ts_ary = df_to_ary(ts_df)  # Test data
vl_ary = df_to_ary(vl_df)  # Validation data

## functions

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, ary, batch_size):
        '''
        Initializes a DataGenerator object.

        Arguments:
        df (pandas.DataFrame): DataFrame containing image paths.
        ary (numpy.ndarray): Array containing keypoints data.
        batch_size (int): Size of the batches for data generation.

        Returns:
        None
        '''
        self.df = df
        self.ary = ary
        self.batch_size = batch_size
        self.n = df['images'].tolist()
        self.on_epoch_end()

    def __len__(self):
        '''
        Calculates the number of batches per epoch.

        Arguments:
        None

        Returns:
        int: Number of batches per epoch.
        '''
        return int(np.floor(len(self.n) / self.batch_size))

    def __getitem__(self, index):
        '''
        Generates a batch of data.

        Arguments:
        index (int): Index of the current batch.

        Returns:
        tuple: Data batch (images, keypoint).
        '''
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        list_IDs_temp = [self.n[k] for k in indexes]
        x, y = self.__data_generation(list_IDs_temp, indexes)
        return x, y

    def on_epoch_end(self):
        '''
        Updates indexes after each epoch.

        Arguments:
        None

        Returns:
        None
        '''
        self.indexes = np.arange(len(self.n))

    def __data_generation(self, list_IDs_temp, indexes):
        '''
        Generates data containing batch_size samples.

        Arguments:
        list_IDs_temp (list): List of image paths.
        indexes (numpy.ndarray): Indexes of the images.

        Returns:
        tuple: Batch of images and keypoints data.
        '''
        images = np.empty((self.batch_size, 768, 768, 1))
        keypoint = np.empty((self.batch_size, 136), dtype = float)

        for i, ID in enumerate(list_IDs_temp):
            # Load and preprocess the image
            train_img = image.img_to_array(image.load_img('F:/poorya/datasets/ThermalFaceDatabase/' + ID, color_mode = "grayscale"))
            train_img /= 255.0
            # Crop the image and assign it to the images array
            images[i,] = train_img[:, 128:896]
            # Assign keypoints to the keypoint array
            keypoint[i,] = self.ary[int(indexes[i]), :] / 768

        return images, keypoint


# Batch size
bs = 4
# Create DataGenerator instances for training, validation, and test data
train_gen = DataGenerator(tr_df, tr_ary, bs)
val_gen = DataGenerator(vl_df, vl_ary, bs)
test_gen = DataGenerator(ts_df, ts_ary, bs)

## model structure

In [ ]:
def get_model():
    '''
    Generates a new model by utilizing transfer learning and adding custom layers.

    Returns:
    tf.keras.Model: The customized model for the specified task.
    '''
    # Initialization
    drp = 0.0
    act = 'relu'

    # Backbone
    img_input = Input(shape = (768, 768, 1))

    # Load pre-trained model and freeze layers
    full_model = load_model('checkpoint')
    model = Model(inputs = full_model.inputs, outputs = full_model.layers[29].output)
    model.trainable = False

    # Connect to the flattened output
    x = model(img_input)

    # Additional Dense layers
    x = Flatten()(x)
    x = Dense(1024, activation = act)(x)
    x = Dropout(drp)(x)
    x = Dense(512, activation = act)(x)
    x = Dropout(drp)(x)
    x = Dense(256, activation = act)(x)
    x = Dropout(drp)(x)

    # Final output layer with sigmoid activation
    last = Dense(136, activation = 'sigmoid')(x)

    return Model(inputs = img_input, outputs = last)


# Define optimizer, loss, and metrics
metrics = ['accuracy']
opt = tf.keras.optimizers.Adamax(learning_rate = 0.0001)
loss = tf.keras.losses.MeanSquaredError()

# Compile the model and summarize its architecture
model = get_model()
model.compile(loss = loss, optimizer = opt, metrics = [tf.keras.metrics.MeanAbsolutePercentageError()])
model.summary()

## train

In [ ]:
def tr():
    '''
    Generates a list of callbacks to use during model training.

    Returns:
    list: List of selected callbacks.
    '''
    # Define different callbacks for model training
    chk = ModelCheckpoint(filepath = 'keypoint final', save_format = "h5", monitor = 'val_loss', mode = 'min', verbose = 1, save_best_only = True)
    ers = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 75)
    rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.8, patience = 5, min_lr = 0.00001)

    # Return a list of selected callbacks (in this case, only the ModelCheckpoint)
    return [chk]


# Obtain the callbacks list by calling the function 'tr()'
cll = tr()

# Fit the model using the generated callbacks along with other parameters
history = model.fit(
    train_gen,
    validation_data = val_gen,
    batch_size = bs,
    epochs = 100,
    verbose = 1,
    callbacks = cll,  # Using the obtained callbacks
    steps_per_epoch = np.shape(tr_ary)[0] // bs,
    validation_steps = np.shape(vl_ary)[0] // bs
    )


## evaluation

In [ ]:
# Retrieve loss and validation loss data from the history object
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

# Plot the data
plt.plot(training_loss, label = 'Training Loss')
plt.plot(validation_loss, label = 'Validation Loss')

# Adding title and labels to the plot
plt.title('Training and Validation Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')

# Display legend and show the plot
plt.legend()
plt.show()

In [ ]:
# Creating an empty image
img = np.zeros((1, 768, 768, 1)).astype('double')

# Loading an image using its path from the DataFrame 'tr_df'
train_igg = image.load_img('F:/poorya/datasets/ThermalFaceDatabase/' + tr_df['images'][40], color_mode = "grayscale")
train_img = image.img_to_array(train_igg)
train_img /= 255.0

# Cropping the image and assigning it to 'img'
img[0, :, :, :] = train_img[:, 128:896]
img = np.array(img)

# Loading the pre-trained model for predicting keypoints
classifier = load_model('keypoint final')
lbl_c = classifier.predict(img)

# Extracting predicted keypoints for x and y axes
x = lbl_c[0, 0:68] * 768
y = lbl_c[0, 68:] * 768

# Plotting the image with the predicted keypoints
plt.figure(figsize = (20, 10))
plt.imshow(img[0], cmap = 'gray')
plt.scatter(y, x)  # Plotting the keypoints
plt.show()

In [ ]:
# Calculate loss and metric for test set
tst_loss, tst_acc = classifier.evaluate(test_gen, steps = np.shape(ts_ary)[0] // bs)

In [ ]:
# Get the list of image paths from the DataFrame
n = ts_df['images'].tolist()

# Create an array to store processed images
x = np.zeros((len(n), 768, 768, 1))

# Process each image and store it in the 'x' array
for i in range(len(n)):
    train_img = image.img_to_array(image.load_img('F:/poorya/datasets/ThermalFaceDatabase/' + n[
        i], color_mode = "grayscale"))
    train_img /= 255.0
    x[i] = train_img[:, 128:896]

# Make predictions using the 'classifier' model for the test images
pred = classifier.predict(x)

# Calculate the Normalized Mean Error (NME)
summ = 0
for i in range(np.shape(pred)[0]):
    y_pred = pred[i] * 768  # Scaling the predicted keypoints
    y_true = ts_ary[i]  # Ground truth keypoints

    # Calculate NME for each keypoint pair (x, y)
    x1 = y_pred[42:48].mean()
    x2 = y_pred[36:42].mean()
    y1 = y_pred[42 + 68:48 + 68].mean()
    y2 = y_pred[36 + 68:42 + 68].mean()

    # Calculate NME for the current image and update the summation
    summ += np.linalg.norm(y_true - y_pred) / (math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2) * 68)

# Calculate the mean NME across all images in the test set
mean_NME = 100 * summ / np.shape(pred)[0]
print(mean_NME)

In [ ]:
summ = []
for i in range(np.shape(pred)[0]):
    y_pred = pred[i] * 768
    y_true = ts_ary[i]

    # Calculate width and height for normalization
    w = max(y_true[0:68]) - min(y_true[0:68])
    h = max(y_true[68:]) - min(y_true[68:])

    # Normalize factor
    ni = 2 / (w + h)

    # Calculate a similarity metric and append it to the 'summ' list
    summ.append(math.sqrt((np.sum(y_true ** 2 + y_pred ** 2)) / 136) * ni)

In [ ]:
# Create a histogram of the 'summ' values with 10 bins
count, bins_count = np.histogram(summ, bins = 10)

# Calculate the Probability Density Function (PDF) using the histogram count values
pdf = count / sum(count)

# Calculate the Cumulative Distribution Function (CDF) using the PDF values
cdf = np.cumsum(pdf)

# Plot the CDF using matplotlib
plt.plot(bins_count[1:], cdf, label = "CDF")